In [114]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings

warnings.filterwarnings('ignore')

- year month date weekends 
- year quater
- parts of month
- public holidays
- events like ramzan , hajj , muhharam 
- paydays


In [115]:
df = pd.read_json(r"C:\Users\Hp\Desktop\cashInfo-1006.json" , lines= True) 
df

,ATMId,Id,RecTime,Type1Count,Type2Count,Type3Count,Type4Count,TotalValue,IsHoliday,HolidayType
0,1006,8211,2020-01-01T00:00:00,22,12,3,5,42,False,None
1,1006,8212,2020-01-02T00:00:00,23,1,6,3,33,False,None
2,1006,8213,2020-01-03T00:00:00,3,38,10,4,55,False,None
3,1006,8214,2020-01-04T00:00:00,24,38,6,4,72,True,SS
4,1006,8215,2020-01-05T00:00:00,4,10,13,9,36,True,SS
...,...,...,...,...,...,...,...,...,...,...
1637,1006,9848,2024-06-25T00:00:00,37,15,2,6,60,False,None
1638,1006,9849,2024-06-26T00:00:00,4,7,16,3,30,False,None
1639,1006,9850,2024-06-27T00:00:00,36,37,5,3,81,False,None
1640,1006,9851,2024-06-28T00:00:00,37,3,12,6,58,False,None


In [116]:
print("DataFrame Information:")
print("-" * 30)
print('DF Shape: ' ,df.shape)
print('number of Columns: ' ,len(df.columns))
print('number of Observations: ' ,len(df))
print('Number of values in df: ' , df.count().sum())
print('Total Number of Missing values in df: ' , df.isna().sum().sum())
print('percentage of Missing values : ' ,  "{:.2f}".format(df.isna().sum().sum()/df.count().sum() *100),'%')
print('Total Number of Duplicated records in df : ' , df.duplicated().sum().sum())
print('percentage of Duplicated values : ' ,  "{:.2f}".format(df.duplicated().sum().sum()/df.count().sum() *100),'%')

DataFrame Information:
------------------------------
DF Shape:  (1642, 10)
number of Columns:  10
number of Observations:  1642
Number of values in df:  15247
Total Number of Missing values in df:  1173
percentage of Missing values :  7.69 %
Total Number of Duplicated records in df :  0
percentage of Duplicated values :  0.00 %


In [117]:
df['RecTime'] = pd.to_datetime(df['RecTime'])
df['Year'] = df['RecTime'].dt.year
df['Month'] = df['RecTime'].dt.month
df['Date'] = df['RecTime'].dt.day
df['DayOfWeek'] = df['RecTime'].dt.dayofweek
df['IsWeekend'] = df['RecTime'].dt.dayofweek >= 5
df['Quarter'] = df['RecTime'].dt.quarter
df['DayOfYear'] = df['RecTime'].dt.dayofyear
df['PartOfMonth'] = df['RecTime'].dt.day.apply(lambda day: '1' if day <= 10 else ('2' if day <= 20 else '3'))


df.drop(['ATMId' , 'Id' , 'HolidayType' ] , axis= 1 , inplace= True)
df.head(10)
df = df.fillna(0)

In [118]:
def convert(t1 = 100 , t2 = 500 , t3 = 1000 , t4 = 5000):
    df['Type1Value'] = t1 * df['Type1Count']
    df['Type2Value'] = t2 * df['Type2Count']
    df['Type3Value'] = t3 * df['Type3Count']
    df['Type4Value'] = t4 * df['Type4Count']
    df['TotalValue'] = df['Type1Value'] + df['Type2Value'] + df['Type3Value'] + df['Type4Value'] 
    

convert ()

In [119]:
usd = pd.read_csv(r"C:\Users\Hp\Downloads\USD_PKR Historical Data (1).csv")
usd['RecTime'] = pd.to_datetime(usd["Date"])
usd

,Date,Price,Open,High,Low,Vol.,Change %,RecTime
0,06/28/2024,278.200,279.100,279.100,278.300,NaN,-0.07%,2024-06-28
1,06/27/2024,278.400,279.075,279.115,278.375,NaN,0.00%,2024-06-27
2,06/26/2024,278.400,279.025,279.100,278.400,NaN,-0.16%,2024-06-26
3,06/25/2024,278.840,279.475,279.475,278.425,NaN,-0.10%,2024-06-25
4,06/24/2024,279.120,279.425,279.475,278.500,NaN,0.40%,2024-06-24
...,...,...,...,...,...,...,...,...
1198,01/06/2020,154.470,154.500,155.480,154.490,0.12K,-0.26%,2020-01-06
1199,01/04/2020,154.875,154.875,154.875,154.875,NaN,0.28%,2020-01-04
1200,01/03/2020,154.450,155.000,155.170,154.490,0.11K,-0.10%,2020-01-03
1201,01/02/2020,154.600,154.870,155.240,154.410,0.15K,0.08%,2020-01-02


In [120]:
def convert_to_usd(date, total_value):
    # Fetch the exchange rate for the given date
    price_row = usd[usd['RecTime'] <= date]
    
    if not price_row.empty:
        # Find the most recent available exchange rate before or on the given date
        latest_price_row = price_row.sort_values(by='RecTime', ascending=False).iloc[0]
        price = latest_price_row['Price']
        # Convert TotalValue to USD
        return total_value / price
    else:
        # If no exchange rate is available, return NaN or handle accordingly
        return None

df['TotalValueUSD'] = df.apply(lambda row: convert_to_usd(row['RecTime'], row['TotalValue']), axis=1)

df['prevWeek'] = df['TotalValueUSD'].rolling(window=7, min_periods=1).mean().shift(1)


In [121]:
public_holidays = {
    '2020-02-05': 'Kashmir Day',
    '2020-03-23': 'Pakistan Day',
    '2020-05-01': 'Labour Day',
    '2020-05-24': 'Eid-ul-Fitr Day 1',
    '2020-05-25': 'Eid-ul-Fitr Day 2',
    '2020-05-26': 'Eid-ul-Fitr Day 3',
    '2020-07-31': 'Eid al-Adha Day 1',
    '2020-08-01': 'Eid al-Adha Day 2',
    '2020-08-02': 'Eid al-Adha Day 3',
    '2020-08-14': 'Independence Day',
    '2020-08-28': 'Ashura',
    '2020-08-29': 'Ashura',
    '2020-10-29': 'Eid Milad un-Nabi',
    '2020-11-09': 'Iqbal Day',
    '2020-12-25': 'Christmas Day',
    '2020-12-25': 'Quaid-e-Azam Day',


    '2021-02-05': 'Kashmir Day',
    '2021-03-23': 'Pakistan Day',
    '2021-05-01': 'Labour Day',
    '2021-05-13': 'Eid-ul-Fitr Day 1',
    '2021-05-14': 'Eid-ul-Fitr Day 2',
    '2021-05-15': 'Eid-ul-Fitr Day 3',
    '2021-07-20': 'Eid al-Adha Day 1',
    '2021-07-21': 'Eid al-Adha Day 2',
    '2021-07-22': 'Eid al-Adha Day 3',
    '2021-08-14': 'Independence Day',
    '2021-08-18': 'Ashura',
    '2021-08-19': 'Ashura',
    '2021-10-19': 'Eid Milad un-Nabi',
    '2021-11-09': 'Iqbal Day',
    '2021-12-25': 'Christmas Day',
    '2021-12-25': 'Quaid-e-Azam Day',
    

    '2022-02-05': 'Kashmir Day',
    '2022-03-23': 'Pakistan Day',
    '2022-05-01': 'Labour Day',
    '2022-05-03': 'Eid-ul-Fitr Day 1',
    '2022-05-04': 'Eid-ul-Fitr Day 2',
    '2022-05-05': 'Eid-ul-Fitr Day 3',
    '2022-07-10': 'Eid-ul-Azha Day 1',
    '2022-07-11': 'Eid-ul-Azha Day 2',
    '2022-07-12': 'Eid-ul-Azha Day 3',
    '2022-08-07': 'Ashura',
    '2022-08-08': 'Ashura',
    '2022-08-14': 'Independence Day',
    '2022-10-09': 'Eid Milad-un-Nabi',
    '2022-12-25': 'Quaid-e-Azam Day',

    '2023-02-05': 'Kashmir Day',
    '2023-03-23': 'Pakistan Day',
    '2023-04-22': 'Eid ul-Fitr Day 1',
    '2023-04-23': 'Eid ul-Fitr Day 2',
    '2023-04-24': 'Eid ul-Fitr Day 3',    
    '2023-05-01': 'Labour Day',
    '2023-06-29': 'Eid ul-Azha Day 1',
    '2023-06-30': 'Eid ul-Azha Day 2',
    '2023-07-1': 'Eid ul-Azha Day 2',
    '2023-07-28': 'Ashura',
    '2023-07-29': 'Ashura',
    '2023-08-14': 'Independence Day',
    '2023-09-28': 'Eid Milad un-Nabi',
    '2023-11-09': 'Iqbal Day',
    '2023-12-25': 'Quaid-e-Azam Day',

    '2024-02-05': 'Kashmir Day',
    '2024-03-23': 'Pakistan Day',
    '2024-05-01': 'Labour Day',
    '2024-05-03': 'Eid ul-Fitr Day 1',
    '2024-05-04': 'Eid ul-Fitr Day 2',
    '2024-05-05': 'Eid ul-Fitr Day 3',
    '2024-07-16': 'Eid ul-Azha Day 1',
    '2024-07-17': 'Eid ul-Azha Day 2',
    '2024-07-18': 'Eid ul-Azha Day 3',
    '2024-08-07': 'Ashura',
    '2024-08-08': 'Ashura',
    '2024-08-14': 'Independence Day',
    '2024-09-28': 'Eid Milad un-Nabi',
    '2024-11-09': 'Iqbal Day',
    '2024-12-25': 'Quaid-e-Azam Day',
    '2024-12-26': 'Day after Christmas',

    '2024-01-01': 'New Year’s Day',
    '2024-02-05': 'Kashmir Day',
    '2024-03-23': 'Pakistan Day',
    '2024-05-01': 'Labour Day',
    '2024-05-03': 'Eid ul-Fitr Day 1',
    '2024-05-04': 'Eid ul-Fitr Day 2',
    '2024-05-05': 'Eid ul-Fitr Day 3'
}
df['HolidayType'] = None
df['IsHoliday'] = False

    # Add 'HolidayType' based on 'RecTime'
df['HolidayType'] = df['RecTime'].apply(lambda x: public_holidays.get(x.strftime('%Y-%m-%d'), None))
df['IsHoliday'] = df['HolidayType'].notna()
df.loc[df['IsWeekend'], 'IsHoliday'] = True


df.loc[df['RecTime'].between(pd.to_datetime('2024-2-8') ,pd.to_datetime('2024-2-9') ), 'HolidayType'] = 'Elections'
df

,RecTime,Type1Count,Type2Count,Type3Count,Type4Count,TotalValue,IsHoliday,Year,Month,Date,...,Quarter,DayOfYear,PartOfMonth,Type1Value,Type2Value,Type3Value,Type4Value,TotalValueUSD,prevWeek,HolidayType
0,2020-01-01,22,12,3,5,36200,False,2020,1,1,...,1,1,1,2200,6000,3000,25000,234.349712,NaN,None
1,2020-01-02,23,1,6,3,23800,False,2020,1,2,...,1,2,1,2300,500,6000,15000,153.945666,234.349712,None
2,2020-01-03,3,38,10,4,49300,False,2020,1,3,...,1,3,1,300,19000,10000,20000,319.197151,194.147689,None
3,2020-01-04,24,38,6,4,47400,True,2020,1,4,...,1,4,1,2400,19000,6000,20000,306.053269,235.830843,None
4,2020-01-05,4,10,13,9,63400,True,2020,1,5,...,1,5,1,400,5000,13000,45000,409.362389,253.386450,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1637,2024-06-25,37,15,2,6,43200,False,2024,6,25,...,2,177,3,3700,7500,2000,30000,154.927557,112.802364,None
1638,2024-06-26,4,7,16,3,34900,False,2024,6,26,...,2,178,3,400,3500,16000,15000,125.359195,122.662098,None
1639,2024-06-27,36,37,5,3,42100,False,2024,6,27,...,2,179,3,3600,18500,5000,15000,151.221264,112.101901,None
1640,2024-06-28,37,3,12,6,47200,False,2024,6,28,...,2,180,3,3700,1500,12000,30000,169.662114,114.794353,None


In [122]:
df[df['HolidayType'].notna()]


,RecTime,Type1Count,Type2Count,Type3Count,Type4Count,TotalValue,IsHoliday,Year,Month,Date,...,Quarter,DayOfYear,PartOfMonth,Type1Value,Type2Value,Type3Value,Type4Value,TotalValueUSD,prevWeek,HolidayType
35,2020-02-05,29,17,13,7,59400,True,2020,2,5,...,1,36,1,2900,8500,13000,35000,387.021110,363.422172,Kashmir Day
82,2020-03-23,22,33,6,3,39700,True,2020,3,23,...,1,83,3,2200,16500,6000,15000,251.584284,310.336213,Pakistan Day
121,2020-05-01,31,20,14,3,42100,True,2020,5,1,...,2,122,1,3100,10000,14000,15000,263.125000,313.593970,Labour Day
144,2020-05-24,3,34,3,0,20300,True,2020,5,24,...,2,145,3,300,17000,3000,0,126.677067,255.596808,Eid-ul-Fitr Day 1
145,2020-05-25,30,4,13,8,58000,True,2020,5,25,...,2,146,3,3000,2000,13000,40000,360.696517,239.236522,Eid-ul-Fitr Day 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1543,2024-03-23,38,33,18,1,43300,True,2024,3,23,...,1,83,3,3800,16500,18000,5000,155.811443,163.571770,Pakistan Day
1582,2024-05-01,32,24,14,8,69200,True,2024,5,1,...,2,122,1,3200,12000,14000,40000,248.920863,152.333768,Labour Day
1584,2024-05-03,36,4,10,9,60600,True,2024,5,3,...,2,124,1,3600,2000,10000,45000,217.946413,164.818257,Eid ul-Fitr Day 1
1585,2024-05-04,2,13,12,8,58700,True,2024,5,4,...,2,125,1,200,6500,12000,40000,211.113109,188.305530,Eid ul-Fitr Day 2


In [123]:
import pandas as pd


df['RecTime'] = pd.to_datetime(df['RecTime'])
ramzan_ranges = [
    (pd.to_datetime('2020-04-24'), pd.to_datetime('2020-05-23')),
    (pd.to_datetime('2021-04-13'), pd.to_datetime('2021-05-12')),
    (pd.to_datetime('2022-04-02'), pd.to_datetime('2022-05-01')),
    (pd.to_datetime('2023-03-22'), pd.to_datetime('2023-04-21')),
    (pd.to_datetime('2024-03-11'), pd.to_datetime('2024-04-09'))
]

hajj_ranges = [
    (pd.to_datetime('2020-07-22'), pd.to_datetime('2020-07-31')),
    (pd.to_datetime('2021-07-11'), pd.to_datetime('2021-07-20')),
    (pd.to_datetime('2022-06-30'), pd.to_datetime('2022-07-09')),
    (pd.to_datetime('2023-06-19'), pd.to_datetime('2023-06-28')),
    (pd.to_datetime('2024-06-07'), pd.to_datetime('2024-06-16'))
]

muharram_ranges = [
    (pd.to_datetime('2020-08-20'), pd.to_datetime('2020-09-18')),
    (pd.to_datetime('2021-08-10'), pd.to_datetime('2021-09-08')),
    (pd.to_datetime('2022-07-31'), pd.to_datetime('2022-08-28')),
    (pd.to_datetime('2023-07-21'), pd.to_datetime('2023-08-19')),
    (pd.to_datetime('2024-07-10'), pd.to_datetime('2024-08-08'))
]



df['Event'] = "Nothing"

for start, end in ramzan_ranges:
    df.loc[df['RecTime'].between(start, end), 'Event'] = 'Ramzan'


for start, end in hajj_ranges:
    df.loc[df['RecTime'].between(start, end), 'Event'] = 'Hajj'
    
for start, end in muharram_ranges:
    df.loc[df['RecTime'].between(start, end), 'Event'] = 'Muharram'



In [124]:
import plotly.graph_objects as go

# Convert 'RecTime' to datetime
df['RecTime'] = pd.to_datetime(df['RecTime'])

# Filter data for the year 2020
df_2020 = df[df['RecTime'].dt.year == 2021]

# Initialize the figure
fig = go.Figure()

# Add a trace for the year 2020
fig.add_trace(go.Scatter(x=df_2020['RecTime'], y=df_2020['TotalValueUSD'], mode='lines', name='2020'))

# Add event annotations using the HolidayType column
events = df_2020[df_2020['HolidayType'].notnull()]

for _, event in events.iterrows():
    fig.add_annotation(
        x=event['RecTime'],
        y=event['TotalValueUSD'],
        text=event['HolidayType'],
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=-30
    )

# Update layout
fig.update_layout(
    title='Total Value in 2020',
    xaxis_title='Date',
    yaxis_title='Total Value',
)

fig.show()


In [125]:
import pandas as pd
from pandas.tseries.offsets import CustomBusinessDay

all_paydays = []

for year in range(2020, 2025):
    paydays_year = []
    
    for month in range(1, 13):
        month_dates = pd.date_range(start=f'{year}-{month:02d}-01', end=f'{year}-{month:02d}-05')
        paydays_year.append(pd.DataFrame({'RecTime': month_dates}))

    paydays_year = pd.concat(paydays_year, ignore_index=True)
    weekmask = 'Mon Tue Wed Thu Fri'
    bday = CustomBusinessDay(weekmask=weekmask)
    paydays_year['RecTime'] = paydays_year['RecTime'] + bday
    
    all_paydays.append(paydays_year)

all_paydays = pd.concat(all_paydays, ignore_index=True)
all_paydays['RecTime'] = pd.to_datetime(all_paydays['RecTime'])
df['Paydays'] = False

# Mark paydays in 'Paydays' column of df
df.loc[df['RecTime'].isin(all_paydays['RecTime']), 'Paydays'] = True

df.head(10)


,RecTime,Type1Count,Type2Count,Type3Count,Type4Count,TotalValue,IsHoliday,Year,Month,Date,...,PartOfMonth,Type1Value,Type2Value,Type3Value,Type4Value,TotalValueUSD,prevWeek,HolidayType,Event,Paydays
0,2020-01-01,22,12,3,5,36200,False,2020,1,1,...,1,2200,6000,3000,25000,234.349712,NaN,None,Nothing,False
1,2020-01-02,23,1,6,3,23800,False,2020,1,2,...,1,2300,500,6000,15000,153.945666,234.349712,None,Nothing,True
2,2020-01-03,3,38,10,4,49300,False,2020,1,3,...,1,300,19000,10000,20000,319.197151,194.147689,None,Nothing,True
3,2020-01-04,24,38,6,4,47400,True,2020,1,4,...,1,2400,19000,6000,20000,306.053269,235.830843,None,Nothing,False
4,2020-01-05,4,10,13,9,63400,True,2020,1,5,...,1,400,5000,13000,45000,409.362389,253.386450,None,Nothing,False
5,2020-01-06,9,6,19,7,57900,False,2020,1,6,...,1,900,3000,19000,35000,374.830064,284.581637,None,Nothing,True
6,2020-01-07,29,11,6,2,24400,False,2020,1,7,...,1,2900,5500,6000,10000,157.847069,299.623042,None,Nothing,False
7,2020-01-08,13,14,1,9,54300,False,2020,1,8,...,1,1300,7000,1000,45000,351.456311,279.369332,None,Nothing,False
8,2020-01-09,38,7,16,7,58300,False,2020,1,9,...,1,3800,3500,16000,35000,378.571429,296.098846,None,Nothing,False
9,2020-01-10,7,23,5,2,27200,False,2020,1,10,...,1,700,11500,5000,10000,176.165803,328.188240,None,Nothing,False


In [126]:
def get_sequence(is_holiday_series):
    sequence = []
    n = len(is_holiday_series)
    for i in range(n):
        if i == 0:  # Only today
            seq = ('H' if is_holiday_series[i] else 'W')
        elif i == 1:  # Yesterday and today
            seq = ('H' if is_holiday_series[i-1] else 'W') + ('H' if is_holiday_series[i] else 'W')
        elif i == n-1:  # Yesterday and today
            seq = ('H' if is_holiday_series[i-1] else 'W') + ('H' if is_holiday_series[i] else 'W')
        elif i == n-2:  # Yesterday, today, and tomorrow
            seq = (
                ('H' if is_holiday_series[i-1] else 'W') +
                ('H' if is_holiday_series[i] else 'W') +
                ('H' if is_holiday_series[i+1] else 'W')
            )
        else:  # Full sequence
            seq = (
                ('H' if is_holiday_series[i-1] else 'W') +
                ('H' if is_holiday_series[i] else 'W') +
                ('H' if is_holiday_series[i+1] else 'W') +
                ('H' if is_holiday_series[i+2] else 'W')
            )
        sequence.append(seq)
    return sequence

df['HolidaySequence'] = get_sequence(df['IsHoliday'])


In [127]:
df.head(10)

,RecTime,Type1Count,Type2Count,Type3Count,Type4Count,TotalValue,IsHoliday,Year,Month,Date,...,Type1Value,Type2Value,Type3Value,Type4Value,TotalValueUSD,prevWeek,HolidayType,Event,Paydays,HolidaySequence
0,2020-01-01,22,12,3,5,36200,False,2020,1,1,...,2200,6000,3000,25000,234.349712,NaN,None,Nothing,False,W
1,2020-01-02,23,1,6,3,23800,False,2020,1,2,...,2300,500,6000,15000,153.945666,234.349712,None,Nothing,True,WW
2,2020-01-03,3,38,10,4,49300,False,2020,1,3,...,300,19000,10000,20000,319.197151,194.147689,None,Nothing,True,WWHH
3,2020-01-04,24,38,6,4,47400,True,2020,1,4,...,2400,19000,6000,20000,306.053269,235.830843,None,Nothing,False,WHHW
4,2020-01-05,4,10,13,9,63400,True,2020,1,5,...,400,5000,13000,45000,409.362389,253.386450,None,Nothing,False,HHWW
5,2020-01-06,9,6,19,7,57900,False,2020,1,6,...,900,3000,19000,35000,374.830064,284.581637,None,Nothing,True,HWWW
6,2020-01-07,29,11,6,2,24400,False,2020,1,7,...,2900,5500,6000,10000,157.847069,299.623042,None,Nothing,False,WWWW
7,2020-01-08,13,14,1,9,54300,False,2020,1,8,...,1300,7000,1000,45000,351.456311,279.369332,None,Nothing,False,WWWW
8,2020-01-09,38,7,16,7,58300,False,2020,1,9,...,3800,3500,16000,35000,378.571429,296.098846,None,Nothing,False,WWWH
9,2020-01-10,7,23,5,2,27200,False,2020,1,10,...,700,11500,5000,10000,176.165803,328.188240,None,Nothing,False,WWHH


In [128]:
df.columns

Index(['RecTime', 'Type1Count', 'Type2Count', 'Type3Count', 'Type4Count',
       'TotalValue', 'IsHoliday', 'Year', 'Month', 'Date', 'DayOfWeek',
       'IsWeekend', 'Quarter', 'DayOfYear', 'PartOfMonth', 'Type1Value',
       'Type2Value', 'Type3Value', 'Type4Value', 'TotalValueUSD', 'prevWeek',
       'HolidayType', 'Event', 'Paydays', 'HolidaySequence'],
      dtype='object')

In [129]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
ft = ['IsHoliday','HolidayType','DayOfWeek','Event' , 'Paydays' , 'HolidaySequence' , 'IsWeekend','PartOfMonth' ]
for i in ft:
    df[i] = le.fit_transform(df[i])

df

,RecTime,Type1Count,Type2Count,Type3Count,Type4Count,TotalValue,IsHoliday,Year,Month,Date,...,Type1Value,Type2Value,Type3Value,Type4Value,TotalValueUSD,prevWeek,HolidayType,Event,Paydays,HolidaySequence
0,2020-01-01,22,12,3,5,36200,0,2020,1,1,...,2200,6000,3000,25000,234.349712,NaN,25,2,0,8
1,2020-01-02,23,1,6,3,23800,0,2020,1,2,...,2300,500,6000,15000,153.945666,234.349712,25,2,1,14
2,2020-01-03,3,38,10,4,49300,0,2020,1,3,...,300,19000,10000,20000,319.197151,194.147689,25,2,1,16
3,2020-01-04,24,38,6,4,47400,1,2020,1,4,...,2400,19000,6000,20000,306.053269,235.830843,25,2,0,11
4,2020-01-05,4,10,13,9,63400,1,2020,1,5,...,400,5000,13000,45000,409.362389,253.386450,25,2,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1637,2024-06-25,37,15,2,6,43200,0,2024,6,25,...,3700,7500,2000,30000,154.927557,112.802364,25,2,0,19
1638,2024-06-26,4,7,16,3,34900,0,2024,6,26,...,400,3500,16000,15000,125.359195,122.662098,25,2,0,19
1639,2024-06-27,36,37,5,3,42100,0,2024,6,27,...,3600,18500,5000,15000,151.221264,112.101901,25,2,0,18
1640,2024-06-28,37,3,12,6,47200,0,2024,6,28,...,3700,1500,12000,30000,169.662114,114.794353,25,2,0,15


In [130]:
y1 = df.pop('Type1Count')
y2 = df.pop('Type2Count')
y3 = df.pop('Type3Count')
y4 = df.pop('Type4Count')


In [131]:
df.drop(['Type1Value', 'Type2Value', 'Type3Value', 'Type4Value'] , inplace= True, axis= 1)

In [132]:
df['TotalValue_diff'] = df['TotalValueUSD'].diff().dropna()


In [133]:
import pandas as pd

# Calculate the correlation matrix
correlation_matrix = df.corr()

# Extract the correlations with TotalValue
correlations_with_total_value = correlation_matrix['TotalValue']

# Extract the correlations with TotalValueUSD
correlations_with_total_value_usd = correlation_matrix['TotalValueUSD']

# Combine correlations into a single DataFrame
correlation_df_total_value = pd.DataFrame({
    'Feature': correlations_with_total_value.index,
    'Correlation with TotalValue': correlations_with_total_value.values,
    'Correlation with TotalValueUSD': correlations_with_total_value_usd.values
})

# Sort the DataFrame by the absolute value of 'Correlation with TotalValue'
correlation_df_total_value = correlation_df_total_value.sort_values(
    by='Correlation with TotalValue', 
    key=lambda x: x.abs(), 
    ascending=False
)

# Display the DataFrame
correlation_df_total_value


,Feature,Correlation with TotalValue,Correlation with TotalValueUSD
0,TotalValue,1.000000,0.837863
10,TotalValueUSD,0.837863,1.000000
16,TotalValue_diff,0.673037,0.587348
14,Paydays,-0.036765,-0.026940
13,Event,-0.029891,-0.013393
11,prevWeek,0.025783,0.448869
2,Year,-0.017950,-0.483489
1,IsHoliday,0.015866,0.020395
5,DayOfWeek,-0.014208,-0.007058
6,IsWeekend,0.010242,0.013273


In [134]:
df.isna().sum()

RecTime            0
TotalValue         0
IsHoliday          0
Year               0
Month              0
Date               0
DayOfWeek          0
IsWeekend          0
Quarter            0
DayOfYear          0
PartOfMonth        0
TotalValueUSD      0
prevWeek           1
HolidayType        0
Event              0
Paydays            0
HolidaySequence    0
TotalValue_diff    1
dtype: int64

In [135]:
df

,RecTime,TotalValue,IsHoliday,Year,Month,Date,DayOfWeek,IsWeekend,Quarter,DayOfYear,PartOfMonth,TotalValueUSD,prevWeek,HolidayType,Event,Paydays,HolidaySequence,TotalValue_diff
0,2020-01-01,36200,0,2020,1,1,2,0,1,1,0,234.349712,NaN,25,2,0,8,NaN
1,2020-01-02,23800,0,2020,1,2,3,0,1,2,0,153.945666,234.349712,25,2,1,14,-80.404046
2,2020-01-03,49300,0,2020,1,3,4,0,1,3,0,319.197151,194.147689,25,2,1,16,165.251485
3,2020-01-04,47400,1,2020,1,4,5,1,1,4,0,306.053269,235.830843,25,2,0,11,-13.143882
4,2020-01-05,63400,1,2020,1,5,6,1,1,5,0,409.362389,253.386450,25,2,0,3,103.309120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1637,2024-06-25,43200,0,2024,6,25,1,0,2,177,2,154.927557,112.802364,25,2,0,19,92.947047
1638,2024-06-26,34900,0,2024,6,26,2,0,2,178,2,125.359195,122.662098,25,2,0,19,-29.568362
1639,2024-06-27,42100,0,2024,6,27,3,0,2,179,2,151.221264,112.101901,25,2,0,18,25.862069
1640,2024-06-28,47200,0,2024,6,28,4,0,2,180,2,169.662114,114.794353,25,2,0,15,18.440849


In [136]:
df = df.rename(columns={'RecTime': 'ds', 'TotalValueUSD': 'y'})
df.drop('TotalValue' , inplace = True , axis=1)

In [137]:
train_size = int(len(df) * 0.8)
df_train = df.iloc[:train_size]
df_test = df.iloc[train_size:]

In [138]:
# import pandas as pd
# import numpy as np
# import statsmodels.api as sm
# from statsmodels.tsa.arima.model import ARIMA
# import plotly.graph_objs as go
# from plotly.subplots import make_subplots
# from sklearn.metrics import mean_absolute_error, mean_squared_error

# # Assuming df has a 'TotalValue_diff' column for differenced data
# # If you haven't differenced the data, do so
# df['TotalValue_diff'] = df['y'].diff().dropna()

# # Split the data into train and test sets
# train_size = int(len(df) * 0.8)
# train, test = df['TotalValue_diff'][:train_size], df['TotalValue_diff'][train_size:]

# # Fit ARIMA model
# model = ARIMA(train, order=(1, 1, 1))
# fitted_model = model.fit()

# # Forecast
# forecast_result = fitted_model.get_forecast(steps=len(test))
# forecast = forecast_result.predicted_mean
# conf_int = forecast_result.conf_int(alpha=0.05)

# # Create a DataFrame to hold the forecasted values and the actual test values
# forecast_df = pd.DataFrame({
#     'actual': test,
#     'forecast': forecast
# })

# # Plotting the results
# fig = make_subplots(rows=1, cols=1)

# # Actual values
# fig.add_trace(go.Scatter(x=test.index, y=test, mode='lines', name='Actual'), row=1, col=1)

# # Forecasted values
# fig.add_trace(go.Scatter(x=test.index, y=forecast, mode='lines', name='Forecast'), row=1, col=1)

# # Confidence intervals
# fig.add_trace(go.Scatter(
#     x=test.index,
#     y=conf_int.iloc[:, 0],
#     mode='lines',
#     line=dict(width=0),
#     showlegend=False
# ), row=1, col=1)

# fig.add_trace(go.Scatter(
#     x=test.index,
#     y=conf_int.iloc[:, 1],
#     mode='lines',
#     fill='tonexty',
#     name='Confidence Interval',
#     line=dict(width=0),
#     showlegend=True
# ), row=1, col=1)

# # Update layout
# fig.update_layout(title='Actual vs Forecasted Values', xaxis_title='Date', yaxis_title='Total Value')

# fig.show()

# # Calculate accuracy metrics
# mae = mean_absolute_error(test, forecast)
# mse = mean_squared_error(test, forecast)
# rmse = np.sqrt(mse)

# print(f'Mean Absolute Error: {mae}')
# print(f'Mean Squared Error: {mse}')
# print(f'Root Mean Squared Error: {rmse}')


In [139]:
# import pandas as pd
# from prophet import Prophet
# from sklearn.metrics import mean_absolute_error, mean_squared_error
# import numpy as np
# model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=True)



# model.fit(df)

# # Make future dataframe for predictions
# future = model.make_future_dataframe(periods=len(df_test))
# forecast = model.predict(future)

# # Extract forecasted values for the test period
# y_pred = forecast['yhat'].iloc[-len(df_test):].values
# y_true = df_test['y'].values

# # Calculate accuracy metrics
# mae = mean_absolute_error(y_true, y_pred)
# mse = mean_squared_error(y_true, y_pred)
# rmse = np.sqrt(mse)

# print(f"Mean Absolute Error (MAE): {mae:.2f}")
# print(f"Mean Squared Error (MSE): {mse:.2f}")
# print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

# print(model.__class__.__name__)

In [140]:
# model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=True)
# model.fit(df_train)

# future = model.make_future_dataframe(periods=len(df_test))

# forecast = model.predict(future)
# y_pred = forecast['yhat'].iloc[-len(df_test):].values


In [141]:
# import plotly.graph_objs as go
# from plotly.subplots import make_subplots

# # Plotting True vs Predicted values using Plotly
# fig = make_subplots(rows=1, cols=1)

# # Add true values trace
# fig.add_trace(
#     go.Scatter(x=df_test['ds'], y=y_true, mode='lines', name='True Values', line=dict(color='blue')),
#     row=1, col=1
# )

# # Add predicted values trace
# fig.add_trace(
#     go.Scatter(x=df_test['ds'], y=y_pred, mode='lines', name='Predicted Values', line=dict(color='red')),
#     row=1, col=1
# )

# # Update layout
# fig.update_layout(
#     title='True vs Predicted Values',
#     xaxis_title='Date',
#     yaxis_title='Values',
#     legend=dict(x=0, y=1.0)
# )

# fig.show()


In [142]:
# from statsmodels.tsa.arima.model import ARIMA

# # Fit ARIMA model
# model = ARIMA(df_train['y'], order=(5, 1, 0))  # (p, d, q)
# model_fit = model.fit()

# # Forecast
# forecast_steps = len(df_test)
# forecast = model_fit.forecast(steps=forecast_steps)
# forecast_index = df_test.index
# forecast_series = pd.Series(forecast, index=forecast_index)

# # Combine actual and forecasted data
# df_forecast = pd.concat([df_train, df_test, forecast_series.rename('y_hat')], axis=1)


In [143]:
df = df.dropna()

In [144]:
# from sklearn.metrics import mean_absolute_error, mean_squared_error

# # Fit ARIMA model
# model = ARIMA(df_train['y'], order=(5, 1, 0))  # (p, d, q)
# model_fit = model.fit()

# # Forecast
# forecast_steps = len(df_test)
# forecast = model_fit.forecast(steps=forecast_steps)
# forecast_series = pd.Series(forecast, index=df_test.index)

# # Calculate accuracy metrics
# mae = mean_absolute_error(df_test['y'], forecast_series)
# mse = mean_squared_error(df_test['y'], forecast_series)
# rmse = np.sqrt(mse)
# # print('MAPE = ' ,mean_absolute_percentage_error(df_test['y'], forecast_series))

# print(f'Mean Absolute Error (MAE): {mae:.2f}')
# print(f'Mean Squared Error (MSE): {mse:.2f}')
# print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')

# # Combine actual and forecasted data for plotting
# df_forecast = pd.concat([df_train, df_test, forecast_series.rename('y_hat')], axis=1)

# # Plot actual vs. forecasted data using Plotly
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=df_train.index, y=df_train['y'], mode='lines', name='Train'))
# fig.add_trace(go.Scatter(x=df_test.index, y=df_test['y'], mode='lines', name='Test'))
# fig.add_trace(go.Scatter(x=forecast_series.index, y=forecast_series, mode='lines', name='Forecast'))

# fig.update_layout(title='ARIMA Model - Actual vs Forecasted',
#                   xaxis_title='Date',
#                   yaxis_title='Value',
#                   legend=dict(x=0.01, y=0.99))

# fig.show()


In [145]:


y = df.pop('y')
df.pop('ds')

from sklearn.model_selection import train_test_split 
xtrain , xtest , ytrain , ytest = train_test_split(df , y , train_size= 0.8)

In [146]:
from sklearn.ensemble import GradientBoostingRegressor

gb = GradientBoostingRegressor(learning_rate =  0.01687770422304368, max_depth = 3, min_samples_leaf = 4, min_samples_split = 19, n_estimators = 409, subsample = 0.8776807051588262)

gb.fit(xtrain  , ytrain)

print(mean_absolute_error(ytest , gb.predict(xtest)))

gb_pred =  gb.predict(xtest)


48.196162905785606


In [147]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.metrics import mean_squared_error
# from scipy.stats import randint, uniform


# # Define the Gradient Boosting Regressor
# gbm = GradientBoostingRegressor()

# # Define the parameter grid
# param_distributions = {
#     'n_estimators': randint(50, 500),
#     'learning_rate': uniform(0.01, 0.2),
#     'max_depth': randint(3, 15),
#     'min_samples_split': randint(2, 20),
#     'min_samples_leaf': randint(1, 20),
#     'subsample': uniform(0.5, 0.5)
# }

# # Set up RandomizedSearchCV
# random_search = RandomizedSearchCV(
#     gbm,
#     param_distributions,
#     n_iter=100,
#     scoring='neg_mean_squared_error',
#     cv=5,
#     verbose=1,
#     random_state=42,
#     n_jobs=-1
# )

# # Fit RandomizedSearchCV
# random_search.fit(xtrain, ytrain)

# # Best parameters
# print("Best Parameters:", random_search.best_params_)

# # Best model
# best_model = random_search.best_estimator_

# # Evaluate the model
# y_pred = best_model.predict(xtest)
# mse = mean_squared_error(ytest, y_pred)
# print("Mean Squared Error on Test Data:", mse)


In [148]:
fig = make_subplots(rows=1, cols=1)

# Add true values trace
fig.add_trace(
    go.Scatter(x=df_test['ds'], y=y, mode='lines', name='True Values', line=dict(color='blue')),
    row=1, col=1
)

# Add predicted values trace
fig.add_trace(
    go.Scatter(x=df_test['ds'], y=gb_pred, mode='lines', name='Tuned Predicted Values', line=dict(color='red', dash='dash')),
    row=1, col=1
)



# Update layout
fig.update_layout(
    title='True vs Predicted Values Using Gradient Boost',
    xaxis_title='Date',
    yaxis_title='Values',
    legend=dict(x=0, y=1.0)
)
print('llll')
fig.show()

llll


In [149]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from xgboost import XGBRegressor
# from sklearn.metrics import mean_squared_error
# from scipy.stats import randint, uniform

# xgb = XGBRegressor()

# # Define the hyperparameter grid
# param_distributions = {
#     'n_estimators': randint(50, 500),
#     'learning_rate': uniform(0.01, 0.2),
#     'max_depth': randint(3, 15),
#     'min_child_weight': randint(1, 10),
#     'subsample': uniform(0.5, 0.5),
#     'colsample_bytree': uniform(0.5, 0.5),
#     'gamma': uniform(0, 1)
# }

# # Set up RandomizedSearchCV
# random_search = RandomizedSearchCV(
#     xgb,
#     param_distributions,
#     n_iter=100,
#     scoring='neg_mean_squared_error',
#     cv=5,
#     verbose=1,
#     random_state=42,
#     n_jobs=-1
# )

# # Fit RandomizedSearchCV
# random_search.fit(xtrain, ytrain)
# 1
# # Best parameters
# print("Best Parameters:", random_search.best_params_)

# # Best model
# best_model = random_search.best_estimator_

# # Evaluate the model
# xg_pred = best_model.predict(xtest)
# mse = mean_squared_error(ytest, y_pred)
# print("Mean Squared Error on Test Data:", mse)


In [150]:
# fig = make_subplots(rows=1, cols=1)

# # Add true values trace
# fig.add_trace(
#     go.Scatter(x=df_test['ds'], y=y, mode='lines', name='True Values', line=dict(color='blue')),
#     row=1, col=1
# )

# # Add predicted values trace
# fig.add_trace(
#     go.Scatter(x=df_test['ds'], y=xg_pred, mode='lines', name='Tuned Predicted Values', line=dict(color='red')),
#     row=1, col=1
# )

# fig.add_trace(
#     go.Scatter(x=df_test['ds'], y=gb_pred, mode='lines', name='Tuned Predicted Values', line=dict(color='yellow')),
#     row=1, col=1
# )

# # Update layout
# fig.update_layout(
#     title='True vs Predicted Values Using Gradient Boost',
#     xaxis_title='Date',
#     yaxis_title='Values',
#     legend=dict(x=0, y=1.0)
# )
# print('llll')
# fig.show()

In [151]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error
# from scipy.stats import randint, uniform

# rf = RandomForestRegressor()

# # Define the hyperparameter grid
# param_distributions = {
#     'n_estimators': randint(50, 500),
#     'max_depth': randint(3, 20),
#     'min_samples_split': randint(2, 20),
#     'min_samples_leaf': randint(1, 20),
#     'bootstrap': [True, False],
#     'max_features': ['auto', 'sqrt', 'log2']
# }

# # Set up RandomizedSearchCV
# random_search = RandomizedSearchCV(
#     rf,
#     param_distributions,
#     n_iter=100,
#     scoring='neg_mean_squared_error',
#     cv=5,
#     verbose=1,
#     random_state=42,
#     n_jobs=-1
# )

# # Fit RandomizedSearchCV
# random_search.fit(xtrain, ytrain)

# # Best parameters
# print("Best Parameters:", random_search.best_params_)

# # Best model
# best_model = random_search.best_estimator_

# # Evaluate the model
# rf_pred = best_model.predict(xtest)
# mse = mean_squared_error(ytest, y_pred)
# print("Mean Squared Error on Test Data:", mse)


In [152]:
# fig = make_subplots(rows=1, cols=1)

# # Add true values trace
# fig.add_trace(
#     go.Scatter(x=df_test['ds'], y=y, mode='lines', name='True Values', line=dict(color='blue')),
#     row=1, col=1
# )

# # Add predicted values trace
# fig.add_trace(
#     go.Scatter(x=df_test['ds'], y=xg_pred, mode='lines', name='Gradient Boost Tuned Predicted Values', line=dict(color='red')),
#     row=1, col=1
# )

# fig.add_trace(
#     go.Scatter(x=df_test['ds'], y=gb_pred, mode='lines', name='XGBoost Tuned Predicted Values', line=dict(color='yellow')),
#     row=1, col=1
# )

# fig.add_trace(
#     go.Scatter(x=df_test['ds'], y=rf_pred, mode='lines', name='RandomForest Tuned Predicted Values', line=dict(color='green')),
#     row=1, col=1
# )


# # Update layout
# fig.update_layout(
#     title='True vs Predicted Values Using Gradient Boost',
#     xaxis_title='Date',
#     yaxis_title='Values',
#     legend=dict(x=0, y=1.0)
# )
# print('llll')
# fig.show()

In [153]:
# from sklearn.metrics import mean_squared_error , mean_absolute_error , mean_absolute_percentage_error , r2_score

# models = {
#     'Random Forest': rf_pred,
#     'Gradient Boost': gb_pred,
#     'XGBoost': xg_pred
# }

# # Calculate and display metrics for each model
# for var_name, predictions in models.items():
#     mse = mean_squared_error(ytest, predictions)
#     mae = mean_absolute_error(ytest, predictions)
#     mape = mean_absolute_percentage_error(ytest, predictions)
#     r2 = r2_score(ytest, predictions)
    
#     print(f"Variable Name: {var_name}")
#     print(f"MSE: {mse:.4f}")
#     print(f"MAE: {mae:.4f}")
#     print(f"MAPE: {mape:.2f}%")
#     print(f"R²: {r2:.4f}")
#     print()


In [154]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


gb = GradientBoostingRegressor(learning_rate =  0.01687770422304368,
    max_depth = 3,min_samples_leaf = 4,
    min_samples_split = 19,n_estimators = 409,
    subsample = 0.8776807051588262)
lgbm= lgb.LGBMRegressor(subsample=0.9,num_leaves=31,
    n_estimators=500,min_child_samples=40,
    learning_rate=0.01,colsample_bytree=0.7
)
cb = CatBoostRegressor(learning_rate=0.01,l2_leaf_reg=3,
    iterations=1000,depth=4,verbose=0
)
rf = RandomForestRegressor(bootstrap=True,max_depth=19,
    max_features='auto',min_samples_leaf=9,
    min_samples_split=13,n_estimators=314
)
xgb = XGBRegressor(colsample_bytree=0.7845407345828739,gamma=0.06351182959000135,
    learning_rate=0.017364373527198277,max_depth=3,
    min_child_weight=5,n_estimators=360,
    subsample=0.9323611881275267
)

pred = []
for i in [gb , lgbm , cb , rf , xgb]:
    i.fit(xtrain , ytrain)
    pred.append(i.predict(xtest))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000253 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 861
[LightGBM] [Info] Number of data points in the train set: 1312, number of used features: 15
[LightGBM] [Info] Start training from score 222.862967
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

In [155]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

# Assuming you have rf_pred, gb_pred, xg_pred, catboost_pred, and lgbm_pred

models = {
    'Gradient Boost',
    'LightGBM',
    'CatBoost',
    'Random Forest',
    'XGBoost'
}

# List to store the evaluation metrics
metrics = []

# Calculate and store metrics for each model
for var_name, predictions in zip(models , pred):
    mse = mean_squared_error(ytest, predictions)
    mae = mean_absolute_error(ytest, predictions)
    mape = mean_absolute_percentage_error(ytest, predictions)
    r2 = r2_score(ytest, predictions)
    
    metrics.append({
        'Model': var_name,
        'MSE': mse,
        'MAE': mae,
        'MAPE': mape,
        'R²': r2
    })

# Create a DataFrame from the metrics
metrics_df = pd.DataFrame(metrics)

# Display the DataFrame
metrics_df


,Model,MSE,MAE,MAPE,R²
0,Gradient Boost,3623.091230,48.283927,0.286228,0.658499
1,XGBoost,3684.105239,47.882289,0.288677,0.652748
2,CatBoost,3438.967833,46.660687,0.281897,0.675853
3,LightGBM,3807.668235,48.943709,0.288999,0.641101
4,Random Forest,3640.756071,48.144883,0.288443,0.656834
